<a href="https://colab.research.google.com/github/marroca13/PSeagrasS2/blob/main/single_image/Planet_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Blending PlanetScope and Sentinel-2 imagery for seagrass change detection**
*Diclaimer: your imagery has to be atmospherically corrected, thats why we integrate ACOLITE processor into the workflow (default settings, v20231023.0). If prefered, you can also use your corrected imagery avoiding step 1 and importing them to Drive or GEE as an asset.*

*This code has been developed for the eutrophic Lagoa da Conceiçao in Florianopolis (Brazil).*

---

The code includes:
1. PlanetScope import
2. ODW and land adjacent pixels masking
3. -
4. Preparation of in situ data: training and validation sets
5. Depth invariant index and band ratios calculation over Sentinel-2
6. Random Forest: binary classification
  6.1. Pre-event + uncertainty assessment
  6.2. Post-event + uncertainty assessment
7. Seagrass change quantification


> **Mar Roca Mora**
Last update: 2025/06/11



## Libraries

In [ ]:
!pip install xlsxwriter
!pip install geemap
!pip install rioxarray
!pip install earthengine-api
!pip install scikit-learn matplotlib

# 0 - Set up

In [ ]:
## Connection to GEE project
import ee
ee.Authenticate()
ee.Initialize(project='your-project') #set your GEE project here

## Connection to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
## Import some libraries
import os
import ee
import geemap
import geemap.colormaps as cm
import pandas as pd
import xarray as xr
import numpy as np
import rioxarray
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
geemap.ee_initialize()

In [ ]:
## Clone GitHub repo:
##!git clone https://github.com/marroca13/seagrass_PS_S2.git



# 1 - Atmospheric correction - ACOLITE inside GEE



## PlanetScope imagery
*If the area has more than one tile, merge first through ACOLITE using merge_tiles = TRUE*

In [ ]:
## Import PlanetScope Imagery already processed and uploaded as an asset

## If you have a Planet account linked to GEE, you can process everything on the cloud
planetScope18 = ee.Image('projects/ee-seagrass-mar/assets/Brazil/PlanetScope_2018') # 4-bands 2259x4305 px
bands18 = ['B1', 'B2', 'B3', 'B4', 'Kd_505', 'KPAR']
planetScope18 = planetScope18.select(planetScope18.bandNames()).rename(['B1', 'B2', 'B3', 'B4', 'flags', 'Kd_505', 'KPAR']).select(bands18)
print(planetScope18.bandNames().getInfo())


planetScope24 = ee.Image('projects/ee-seagrass-mar/assets/Brazil/PlanetScope_2024') # 8-bands 2259x4305 px
bands24 = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'Kd_492', 'KPAR'] # ['B2', 'B4', 'B6', 'B8', 'Kd_492', 'KPAR']
planetScope24 = planetScope24.select(planetScope24.bandNames()).rename(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'flags', 'Kd_492', 'KPAR']).select(bands24)
print(planetScope24.bandNames().getInfo())

In [ ]:
rgb_plot_ps = {
    'bands': ['B3', 'B2', 'B1'],
    'gamma': 1.4,
    'min': -0.003069632846763851,
    'max': 0.023891219740272425}

rgb_plot_ps_24 = {
    'bands': ['B6', 'B4', 'B2'],
    'min': 0.0017,
    'max': 0.031,
    'gamma': 1.4
}

Map = geemap.Map()
Map.addLayer(planetScope18, rgb_plot_ps, "PlanetScope 2018")
Map.addLayer(planetScope24, rgb_plot_ps_24, "PlanetScope 2024")
Map.centerObject(planetScope18, 11)
Map


# 2 - Optically deep water (ODW) and land pixel masking

In [ ]:
# For the ODW masking we used a delimitation from 1-meter secchi disk for 2024 processed for Sentinel-2 imagery.
OSW = ee.FeatureCollection('projects/ee-seagrass-mar/assets/Brazil/OSW_zSD_new')
bathy = ee.Image('projects/ee-seagrass-mar/assets/Brazil/Batimetria_Lagoa')

In [ ]:
ndwi24 = planetScope24.normalizedDifference(['B8', 'B4'])
land_mask = ndwi24.lt(0.1)

Map.addLayer(land_mask, {'palette': 'orange'}, "Land Mask 24")
# Map.addLayer(OSW, {'color': 'yellow'}, "OSW")
# Map.centerObject(land_mask, 11)
# Map

zSD = ee.Image('projects/ee-seagrass-mar/assets/Brazil/PlanetScope_24').select('zSD')
zSD = zSD.updateMask(zSD.lte(1))
land_mask = ndwi24.lt(0.1).And(zSD.lte(1))
Map.addLayer(land_mask, {'palette': 'orange'}, "Shallow water mask")
# Map.centerObject(land_mask, 11)
# Map

Map.addLayer(zSD, {"min":0,"max":1,"palette":["e0f7fa","b2ebf2","81d4fa","4fc3f7","29b6f6","039be5","0288d1","0277bd","01579b","003f5c"]}, "zSD")
Map.centerObject(zSD, 11)
Map

In [ ]:

planetScope18 = planetScope18.clip(OSW).updateMask(land_mask).select(['B1', 'B2', 'B3', 'B4', 'Kd_505', 'KPAR']) ## try remove B4 (NIR)
planetScope24 = planetScope24.clip(OSW).updateMask(land_mask).select(['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'Kd_492', 'KPAR']) ## try remove B8 (NIR)

Map.centerObject(land_mask, 11)
Map

# 4 - Preparation of *in situ* data: training and validation sets

Field sample data have the following classes as 'Bottom':

 0 - seagrass (Halodule or Ruppia)

 1 - nonSeagrass (sand, mud, rock, algae)

In [ ]:

insitu2018 = ee.FeatureCollection('projects/ee-seagrass-mar/assets/Brazil/fieldData18').filterBounds(OSW)
sand18 = insitu2018.filter(ee.Filter.eq('Bottom', 'sand'))
# print('Total samples 2018:', insitu2018.size().getInfo())
# print('Sand sample 2018:',sand18.size().getInfo())

# in situ 2024 has 6 different bottom types: sand, mud, rock, algae, Halodule and Ruppia species, so we'll group them later ['Halodule, 'Ruppia'] == seagrass
insitu2024 = ee.FeatureCollection('projects/ee-seagrass-mar/assets/Brazil/fieldData24').filterBounds(OSW)

# Filter for sand class and get size on the server side
sand24 = insitu2024.filter(ee.Filter.eq('Bottom', 'sand'))
# seagrass24 = insitu2024.filter(ee.Filter.inList('Bottom', ['Halodule', 'Ruppia'])) # Keep this commented out as it was before
# print('Total samples 2024:', insitu2024.size().getInfo())
# print('Sand sample 2024:', sand24.size().getInfo())
# print('Seagrass sample 2024:',seagrass24.size().getInfo()) # Keep this commented out as it was before

In [ ]:
Map.addLayer(insitu2018, {'color': 'darkgreen'}, 'in situ 2018')
Map.addLayer(insitu2024, {'color': 'lightgreen'}, 'in situ 2024')
# Map.addLayer(land_mask, {'color': 'blue'}, 'secchi_mask')
Map

In [ ]:
# Map the sampling function to the ImageCollection
sampledData_2018 = planetScope18.sampleRegions(collection=insitu2018,
        properties=['Bottom'],
        scale=3,
        geometries=True)

# print('sampledData 2018:', sampledData_2018.size().getInfo())

# Map the sampling function to the ImageCollection
sampledData_2024 = planetScope24.sampleRegions(
        collection=insitu2024,
        properties=['Bottom'],
        scale=3,
        geometries=True)

sampledData_2024
sampledData_2018

In [ ]:
## Split training and validation sets ## Be sure in a Random Forest that the dataset is balanced (50%/50% of each class)
## 2018
filtered_18_seagrass = insitu2018.filter(ee.Filter.eq('Bottom', 'seagrass')).randomColumn('random')
filtered_18_nonSeagrass = insitu2018.filter(ee.Filter.eq('Bottom', 'sand')).randomColumn('random')

# Get the count of seagrass and sand points in filtered_18
seagrass_samples_18 = filtered_18_seagrass.limit(90)
nonSeagrass_samples_18 = filtered_18_nonSeagrass.limit(90)

seagrass_count_18 = seagrass_samples_18.size().getInfo()
nonSeagrass_count_18 = nonSeagrass_samples_18.size().getInfo()

# Create labels and counts for the bar chart
labels = ['Seagrass', 'Non-Seagrass']
counts = [seagrass_count_18, nonSeagrass_count_18]

# Create the bar chart
plt.figure(figsize=(6, 4))
plt.bar(labels, counts, color=['green', 'darkorange'])
plt.ylabel('Number of Points')
plt.title('Distribution of Seagrass vs. Sand Points in filtered_18')
plt.show()

## 2024
filtered_24_seagrass = insitu2024.filter(ee.Filter.inList('Bottom', ['Hadolule', 'Ruppia'])).randomColumn('random')
filtered_24_nonSeagrass = insitu2024.filter(ee.Filter.inList('Bottom', ['sand',  'mud'])).randomColumn('random')

## 2024
# Separate features by class groupings
seagrass_samples_24 = filtered_24_seagrass.filter(ee.Filter.inList('Bottom', ['Hadolule', 'Ruppia'])).limit(60)
nonSeagrass_samples_24 = filtered_24_nonSeagrass.filter(ee.Filter.inList('Bottom', ['sand',  'mud', 'rock', 'algae'])).limit(60) # Adjust this list based on your non-seagrass classes

# Get the count of seagrass and non-seagrass points in filtered_24
seagrass_count_24 = seagrass_samples_24.size().getInfo()
nonSeagrass_count_24 = nonSeagrass_samples_24.size().getInfo()

# Create labels and counts for the bar chart
labels = ['Seagrass', 'Non-Seagrass']
counts = [seagrass_count_24, nonSeagrass_count_24]

# Create the bar chart
plt.figure(figsize=(6, 4))
plt.bar(labels, counts, color=['green', 'darkorange'])
plt.ylabel('Number of Points')
plt.title('Distribution of Seagrass vs. Non-Seaglass Points in filtered_24')
plt.show()

In [ ]:
## Split training and validation sets
## 2018
t_seagrass_pre = seagrass_samples_18.filter(ee.Filter.eq('Bottom', 'seagrass')).filter(ee.Filter.lt('random', 0.7))
v_seagrass_pre = seagrass_samples_18.filter(ee.Filter.eq('Bottom', 'seagrass')).filter(ee.Filter.gte('random', 0.7))
t_nonSeagrass_pre = nonSeagrass_samples_18.filter(ee.Filter.eq('Bottom', 'sand')).filter(ee.Filter.lt('random', 0.7))
v_nonSeagrass_pre = nonSeagrass_samples_18.filter(ee.Filter.eq('Bottom', 'sand')).filter(ee.Filter.gte('random', 0.7))
validation_pre = v_seagrass_pre.merge(v_nonSeagrass_pre)
print('Training pre:', t_seagrass_pre.merge(t_nonSeagrass_pre).size().getInfo())
print('t_seagrass_pre:', t_seagrass_pre.size().getInfo())
print('t_nonSeagrass_pre:', t_nonSeagrass_pre.size().getInfo())
print('Validation pre:', validation_pre.size().getInfo())
print('v_seagrass_pre:', v_seagrass_pre.size().getInfo())
print('v_nonSeagrass_pre:', v_nonSeagrass_pre.size().getInfo())

## 2024
t_seagrass_post = seagrass_samples_24.filter(ee.Filter.inList('Bottom', ['Hadolule', 'Ruppia'])).filter(ee.Filter.lt('random', 0.7))
v_seagrass_post = seagrass_samples_24.filter(ee.Filter.inList('Bottom', ['Hadolule', 'Ruppia'])).filter(ee.Filter.gte('random', 0.7))
t_nonSeagrass_post = nonSeagrass_samples_24.filter(ee.Filter.inList('Bottom', ['sand',  'mud'])).filter(ee.Filter.lt('random', 0.7)) ##['sand', 'rock', 'mud', 'algae']))
v_nonSeagrass_post = nonSeagrass_samples_24.filter(ee.Filter.inList('Bottom', ['sand', 'mud'])).filter(ee.Filter.gte('random', 0.7)) ## Try including only sand and mud
validation_post = v_seagrass_post.merge(v_nonSeagrass_post)
print('Training post:', t_seagrass_post.merge(t_nonSeagrass_post).size().getInfo())
print('t_seagrass_post:', t_seagrass_post.size().getInfo())
print('t_nonSeagrass_post:', t_nonSeagrass_post.size().getInfo())
print('Validation post:', v_seagrass_post.merge(v_nonSeagrass_post).size().getInfo())
print('v_seagrass_post:', v_seagrass_post.size().getInfo())
print('v_nonSeagrass_post:', v_nonSeagrass_post.size().getInfo())

# 5 - Depth Invariant Index (DII) - B2B3 for both sensors

In [ ]:
# Rename bands for PlanetScope
ps_prefix = 'PS_'
planetScope18 = planetScope18.rename(planetScope18.bandNames().map(lambda band: ee.String(ps_prefix).cat(band)))
# Rename bands for PlanetScope
ps_prefix = 'PS_'
planetScope24 = planetScope24.rename(planetScope24.bandNames().map(lambda band: ee.String(ps_prefix).cat(band)))
planetScope18

In [ ]:
# For PlanetScope Classic
def PSC_depthInvariant(image):
  band1 = ['PS_B1', 'PS_B2', 'PS_B1']
  band2 = ['PS_B2', 'PS_B3', 'PS_B3']
  nband = ['PS_B1B2', 'PS_B2B3', 'PS_DII']

  for i in range(3):
    x = band1[i]
    y = band2[i]
    z = nband[i]

    imageLog = image.select([x, y]).log()

    sand = insitu2018.filter(ee.Filter.eq('Bottom', 'sand'))

    covariance = imageLog.toArray().reduceRegion(
        reducer=ee.Reducer.covariance(),
        geometry=sand,
        scale=3,
        maxPixels=1e13,
        bestEffort=True
    )

    covarMatrix = ee.Array(covariance.get('array'))
    var1 = covarMatrix.get([0, 0])
    var2 = covarMatrix.get([1, 1])
    covar = covarMatrix.get([0, 1])

    a = (var1.subtract(var2)).divide(covar.multiply(2))
    attenCoeffRatio = a.add(((a.pow(2)).add(1)).sqrt())

    depthInvariantIndex = image.expression(
        'image1 - (image2 * coeff)', {
            'image1': imageLog.select([x]),
            'image2': imageLog.select([y]),
            'coeff': attenCoeffRatio
        }
    )

    image = image.addBands(depthInvariantIndex.select([x], [z])) # Corrected index to 0 as expression returns a single band

  return image

# For PlanetScope SuperDove
def PSSD_depthInvariant(image):
  band1 = ['PS_B2', 'PS_B3', 'PS_B2']
  band2 = ['PS_B3', 'PS_B4', 'PS_B4']
  nband = ['PS_B2B3', 'PS_B3B4', 'PS_DII']

  for i in range(3):
    x = band1[i]
    y = band2[i]
    z = nband[i]

    imageLog = image.select([x, y]).log()

    sand = insitu2024.filter(ee.Filter.eq('Bottom', 'sand'))

    covariance = imageLog.toArray().reduceRegion(
        reducer=ee.Reducer.covariance(),
        geometry=sand,
        scale=3,
        maxPixels=1e13,
        bestEffort=True
    )

    covarMatrix = ee.Array(covariance.get('array'))
    var1 = covarMatrix.get([0, 0])
    var2 = covarMatrix.get([1, 1])
    covar = covarMatrix.get([0, 1])

    a = (var1.subtract(var2)).divide(covar.multiply(2))
    attenCoeffRatio = a.add(((a.pow(2)).add(1)).sqrt())

    depthInvariantIndex = image.expression(
        'image1 - (image2 * coeff)', {
            'image1': imageLog.select([x]),
            'image2': imageLog.select([y]),
            'coeff': attenCoeffRatio
        }
    )

    image = image.addBands(depthInvariantIndex.select([x], [z])) # Corrected index to 0 as expression returns a single band

  return image

# Apply to your image collection (e.g., multi_sensor_pre or multi_sensor_post)
multi_sensor_pre = PSC_depthInvariant(planetScope18)
multi_sensor_post = PSSD_depthInvariant(planetScope24)
multi_sensor_post

# 6 - Binary classification

## 6.1 - Pre-event Multi-sensor Image Collection
#### Before processing all data, the best seagrass probability threshold was studied and set as t = 45

In [ ]:
# Binary classification setup
seagrass_class = 0
nonSeagrass_class = 1

# Classification and validation parameters
trees = 50
t = 45

# Boxcar kernel: a square matrix where all values are equal, and it calculates the mean of the pixel values within that square neighborhood.
boxcar = ee.Kernel.square(radius=2, units='pixels', normalize=True)
bands = multi_sensor_pre.bandNames()

def boxcar_image(image):
    return image.convolve(boxcar)

# Apply the function to every image in the collection
classificationComp_pre = boxcar_image(multi_sensor_pre)
classificationComp_pre

 ##### 6.1.1 - Prepare training dataset

In [ ]:
# Training data - class property recoding and merging - MODEL 1 (PRE-EVENT)
t_seagrass = t_seagrass_pre.map(lambda x: x.setMulti(ee.Dictionary.fromLists(['habitat'], [seagrass_class])))
t_nonSeagrass = t_nonSeagrass_pre.map(lambda x: x.setMulti(ee.Dictionary.fromLists(['habitat'], [nonSeagrass_class])))
t_FC = t_seagrass.merge(t_nonSeagrass)


# Collect sampled features from each image
def sample_image(image):
    sampled = image.sampleRegions(
        collection=t_FC,
        scale=3,
        geometries=True
    ).filter(ee.Filter.notNull(bands))

    return sampled

# Map the sampling function and convert result to a list of FeatureCollections
sampledData = sample_image(classificationComp_pre)

# Using sampleRegions we take values across the Image Collection. getting valid FeatureCollection when overlapping with pixels
print('sampledData:', sampledData.size().getInfo())
print('t_FC:', t_FC.size().getInfo())

6.1.2 - Extract probabilities for each class and train classifier

In [ ]:
## To extract feature importance and evaluate
def soft_prob_subfn(image, num):
       training = sampledData.map(lambda ft: ft.set(
        'prob',
        ee.Algorithms.If(ft.getNumber('habitat').eq(num), 1, 0)
        ))

       trained = ee.Classifier.smileRandomForest(numberOfTrees=trees) \
           .train(training, 'prob', image.bandNames()) \
           .setOutputMode('PROBABILITY')

       dict_classifier = trained.explain()

       # Create a Feature with the dictionary as a property
       feature = ee.Feature(None, {'classifier_explanation': dict_classifier})

       # Classify and convert to percentage scale
       classified = image.classify(trained).multiply(100).toInt8()

       # Return both the classified image and the Feature with explanation
       return ee.Image(classified).set('classifier_explanation', feature)

# Apply the function to the single image
seagrass_prob_image = soft_prob_subfn(classificationComp_pre, seagrass_class)
nonSeagrass_prob_image = soft_prob_subfn(classificationComp_pre, nonSeagrass_class)

# You can now work with seagrass_prob_image and nonSeagrass_prob_image

### FEATURE IMPORTANCE
seagrass_prob_image

# # Initialize an empty dictionary to store feature importances
# mean_importance = {}

# # Iterate through the first three images
# for i in range(1):
#   image = ee.Image(seagrass_prob_image.get(i))
#   classifier_explanation = image.get('classifier_explanation').getInfo()['properties']['classifier_explanation']
#   importance_dict = classifier_explanation.get('importance')

#   # Accumulate feature importances
#   for feature, importance in importance_dict.items():
#     mean_importance[feature] = mean_importance.get(feature, 1) + importance

# # Calculate the mean feature importance
# for feature in mean_importance:
#   mean_importance[feature]

# # Print the mean feature importance
# print(mean_importance)

# # Sort features by importance in descending order
# sorted_importance = dict(sorted(mean_importance.items(), key=lambda item: item[1], reverse=False))

# # Extract feature names and importance values
# feature_names = list(sorted_importance.keys())
# importance_values = list(sorted_importance.values())

# # Create a list of colors based on feature name prefix
# colors = ['#005b96' if name.startswith('PS_') else '#6497b1' for name in feature_names]

# # Create the horizontal bar graph (switched axes)
# plt.figure(figsize=(10, 6))  # Adjust figure size if needed
# plt.barh(feature_names, importance_values, color=colors) # Changed to barh for horizontal bars
# plt.ylabel("Features") # Switched x and y labels
# plt.xlabel("Importance") # Switched x and y labels
# plt.title("Feature Importance Histogram (2018)")
# # plt.axvline(x=10.5, linestyle='dotted', color='grey')  # Add dotted vertical line
# plt.show()  # Display the plot
# plt.tight_layout()

# selected_bands = [band for band, importance in mean_importance.items() if importance >= 0] ## Use only selected bands for the classification ## 10.5 before

# print("Selected bands:", selected_bands)


Analyse covariance between variables

In [ ]:
## Co-variance analysis
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

selected_bands = classificationComp_pre.bandNames()

# Function to extract band values from a FeatureCollection
def extract_band_values(feature_collection, bands):
    data = feature_collection.toList(feature_collection.size()).map(lambda feature: ee.Feature(feature).toDictionary().select(bands)).getInfo()
    df = pd.DataFrame(data)
    return df

# Extract band values for selected bands
band_values_df = extract_band_values(sampledData, selected_bands)

# Calculate the correlation matrix
correlation_matrix = band_values_df.corr()

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, bool), k=1)

# Apply the mask to the filtered correlation matrix
filtered_correlation_matrix = correlation_matrix.mask(mask)
filtered_correlation_matrix = filtered_correlation_matrix[np.abs(correlation_matrix) > 0.80]

# Get bands with at least one correlation above 0.9
bands_to_keep = filtered_correlation_matrix.columns[filtered_correlation_matrix.any()]

# Filter the original correlation matrix to keep only relevant bands
filtered_correlation_matrix = correlation_matrix.loc[bands_to_keep, bands_to_keep]

# Plot only the lower triangle of the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(filtered_correlation_matrix, mask=mask, annot=True, cmap='coolwarm',
            fmt=".2f", linewidths=.5, linecolor='lightgrey', square=True,
            cbar_kws={"shrink": .75})
plt.title('Band Covariability Analysis')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("Bands to keep:", bands_to_keep.tolist())

6.1.3 - Generate probability layers

In [ ]:
## Train classifier and display probability map for seagrasses
# Define classes dictionary
classes = {
  'classes_values': [seagrass_class, nonSeagrass_class],
  'classes_names': ['seagrass','nonSeagrass'] # Corrected 'nonSeagrass' spelling
}

classes

# Assume this is already defined:
# - classificationComp_pre: an ImageCollection
# - classes: dictionary with class values and names
# - soft_prob_subfn(image, class_id): returns a probability image for one class

def classify_single_image_with_all_classes(image):
    """
    Classifies a single Earth Engine Image for all defined classes.

    Args:
        image (ee.Image): The Earth Engine Image to classify.

    Returns:
        ee.Image: A multi-band image with probability bands for each class.
    """
    # Classify image for each class and rename the result accordingly
    classified_images = ee.List(classes['classes_values']).map(
        lambda class_id: soft_prob_subfn(image, class_id)
    )

    # Convert list of images into one multi-band image (each class as band)
    classified_combined = ee.ImageCollection(classified_images).toBands()
    classified_combined = classified_combined.rename(classes['classes_names'])

    # Preserve metadata if needed
    return classified_combined.copyProperties(image, image.propertyNames())

probabilities = classify_single_image_with_all_classes(classificationComp_pre)
probabilities
# You can now use probabilities_single_image for visualization or further analysis
# print(probabilities_single_image.bandNames().getInfo())

Plot probabilities:

In [ ]:
# Visualize probabilities
probabilities = ee.Image(probabilities).clip(OSW).updateMask(land_mask)
Map = geemap.Map()
Map.addLayer(probabilities.select('seagrass'), {'min': 45, 'max': 100}, 'Seagrass Prob')
Map.addLayer(probabilities.select('nonSeagrass'), {'min': 45, 'max': 100}, 'NonSeagrass Prob')
Map.centerObject(probabilities, zoom=12)
Map

6.1.4 - Threshold classes based on probabilities and classify

In [ ]:
# Based on the generated probabilities, apply thresholding for seagrass and nonSeagrass on the whole Image Collection

# Threshold function for seagrass
def threshold_seagrass(image):
    return image.select('seagrass').updateMask(image.select('seagrass').gt(t)) \
        .copyProperties(image, image.propertyNames())

# Threshold function for non-seagrass
def threshold_non_seagrass(image):
    return image.select('nonSeagrass').updateMask(image.select('nonSeagrass').gt(t)) \
        .copyProperties(image, image.propertyNames())

# Map the thresholding over the probabilities collection
seagrass_soft = threshold_seagrass(probabilities)
nonSeagrass_soft = threshold_non_seagrass(probabilities)

# Combine the thresholded images into a single image (taking the first image from each collection as an example)
# If you intend to process all images in the collection, the downstream code needs to be adapted to work with ImageCollections.
soft_map = ee.Image(seagrass_soft).addBands(ee.Image(nonSeagrass_soft))
soft_map

6.1.5 - Perform classification

In [ ]:
def soft_to_hard(image):
    seagrass = image.select('seagrass')
    non_seagrass = image.select('nonSeagrass')
    prob_aoi = ee.Image(probabilities).geometry()

    classified = (ee.Image.constant(2)
        .where(seagrass.gte(t), ee.Number(0).add(1))  # temporary +1 to mask 0s
        .where(non_seagrass.gte(t), ee.Number(1).add(1))
        .selfMask()
        .subtract(1)  # back to 0 and 1
        .rename('classification')
        .clip(prob_aoi))
    return classified

# Apply to soft_map
soft_map_th = soft_to_hard(soft_map)
soft_map_th

6.1.6 - Combine the three classifications for visualization

In [ ]:
# 1. Extract the 'classification' band from each image
classification_images_pre = soft_map_th.select('classification').mask(land_mask).clip(OSW)

# Display the combined classification
Map.addLayer(classification_images_pre, {'min': 0, 'max': 1, 'palette': ['42762f', '7cb75a', 'e2e9ab', 'ffffff']}, 'Combined Classification')
Map

In [ ]:
# # Export the FeatureCollection to Google Drive
# task = ee.batch.Export.image.toDrive(
#     image=combined_classification_pre,
#     description='combined_classification_pre',  # Description for the task
#     folder='Colab_Notebooks/export',
#     region = OSW.geometry(),
#     scale=3,  # Adjust the scale as needed
#     fileFormat='GeoTIFF'  # Choose your desired file format (CSV, GeoJSON, KML, SHP, TFRecord)
# )

# # Start the export task
# task.start()

# print('Export task started. You can monitor its progress in the "Tasks" tab.')

In [ ]:
# Apply conditional classification for obtain the final binary map
classification_images_pre = classification_images_pre.expression(
    'b(0) < 0.35 ? 0 : 1',  # Conditional expression
    {'classification': classification_images_pre}  # Band name mapping
).rename('classification')  # Rename the band

# Display the combined classification
Map.addLayer(classification_images_pre.mask(land_mask).clip(OSW.geometry()), {'min': 0, 'max': 1, 'palette': ['42762f', 'ffffff']}, 'Combined Classification')
Map

6.1.7 - Validation and accuracy assessment for each of the 3 classifications

In [ ]:
# Validation function
def valid_habitat(number):
  def wrap(feature):
    return feature.setMulti(ee.Dictionary.fromLists(['habitat'], [number]))
  return wrap

## Validation data
v_seagrass = v_seagrass_pre.map(valid_habitat(seagrass_class))
v_nonSeagrass = v_nonSeagrass_pre.map(valid_habitat(nonSeagrass_class))
v_FC = v_seagrass.merge(v_nonSeagrass)
v_FC

In [ ]:
# Collect sampled features from each image
def validation(image):
    # Buffer the validation points by 10 meters (optional, keep if desired)
   # buffered_v_FC = v_FC.map(lambda feature: feature.buffer(5))

    sampled = image.sampleRegions(
        collection=v_FC,
        properties=['habitat'],
        scale=3
        )
    return sampled


# Map the sampling function and convert result to a list of FeatureCollections
sampled_v_list = validation(soft_map_th)

# Flatten the FeatureCollections into one big FeatureCollection
sampledValidation_pre = ee.FeatureCollection(sampled_v_list) #.filter(ee.Filter.notNull(['habitat']))

## To validate with the final combination of the 3 classifications (this will be better)
# sampledValidation_pre = final_classification_pre.sampleRegions(
#     collection=v_FC,
#     scale=3,
#     properties=['habitat'])

# Using sampleRegions we take values across the Image Collection. getting valid FeatureCollection when overlapping with pixels
print('sampled Validation:', sampledValidation_pre.size().getInfo())
sampledValidation_pre

In [ ]:
errorMatrix = sampledValidation_pre.errorMatrix('habitat', 'classification')
print('Confusion Matrix:', errorMatrix.getInfo())

# Get confusion matrix data as a list of lists
confusion_matrix_data = errorMatrix.array().getInfo()

# Convert the list to a NumPy array
matrix_values = np.array(confusion_matrix_data)  # Convert to NumPy array

# Define class labels
class_names = ['seagrass', 'nonSeagrass']  # Adjust if your class names are different

# Create ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(confusion_matrix=matrix_values, display_labels=class_names)

# Plot the confusion matrix
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.xlabel("EO prediction", fontsize = 14)  # Change to your desired text
plt.ylabel("In situ data", fontsize = 14)
disp.ax_.tick_params(axis='both', which='major', labelsize=18)  # Increase axis tick label size  # Change to your desired text
plt.show()

# Print accuracy metrics
overall_accuracy = errorMatrix.accuracy().getInfo()
print('Overall Accuracy:', overall_accuracy)
producers_accuracy = ee.Array(errorMatrix.producersAccuracy()).reshape([-1]).getInfo()
print('PA:', producers_accuracy)
users_accuracy = ee.Array(errorMatrix.consumersAccuracy()).reshape([-1]).getInfo()
print('UA:', users_accuracy)

# Calcuate F1-score
# Extract the components from the confusion matrix
truePositives = errorMatrix.array().get([0, 0])
falsePositives = errorMatrix.array().get([1, 0])
falseNegatives = errorMatrix.array().get([0, 1])

# Calculate precision, recall, and F1-score
precision = truePositives.divide(truePositives.add(falsePositives))
recall = truePositives.divide(truePositives.add(falseNegatives))
f1_score = ee.Number(2).multiply(precision.multiply(recall)).divide(precision.add(recall))

print('F1-score:', f1_score.getInfo())

### Area quantification and +-

In [ ]:
# Mask seagrass pixels
seagrass_pre = classification_images_pre.eq(0)

# create a binary mask: 1 = seagrass, 0 = not
seagrass_mask = seagrass_pre.rename('seagrass')  # change eq(0) if your seagrass label differs

# count seagrass pixels (sum of 1s)
pixel_count = seagrass_mask.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=OSW.geometry(),
    scale=3, # 3 m pixels
    maxPixels=1e13
).get('seagrass').getInfo()

# convert to hectares: each pixel = 9 m2 -> 9/10000 = 0.0009 ha
area_ha_pre = pixel_count * 9.0 / 10000.0

print(f"Seagrass pixels: {pixel_count}, area: {area_ha_pre} ha")

In [ ]:
# Scale probabilities from 0-100 to 0-1
probabilities_pre = probabilities.divide(100).select('seagrass')

# Get the mean probability of seagrass over the region of interest
mean_sum_prob_seagrass_pre = probabilities_pre.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=OSW.geometry(),
    scale=3, # Use the same scale as the area calculation
    maxPixels=1e13
).get('seagrass') # Get the mean value for the 'seagrass' band

overall_accuracy_pre = ee.Number(overall_accuracy) # Use the overall_accuracy from the pre-event validation cell

# Get the total seagrass area for the pre-event classification (assuming it's stored in area_ha variable from cell ZfHt_UQ49oKf)
# If not, you might need to recalculate it here or fetch it from the appropriate variable.
total_seagrass_area_pre = ee.Number(area_ha_pre)

# Calculate uncertainty for the pre-event seagrass area using the provided formula:
# uncertainty = area * Overall Accuracy * (3 - mean of the sum of the probabilities predicted as seagrass) / 3
# Assuming '3' is the number of images used in the pre-event period (sentinel2_l1c_pre has 3 images)
num_images_pre = ee.Number(1) # Set the number of pre-event images to 1 as an Earth Engine Number

uncertainty_pre_seagrass_area = total_seagrass_area_pre.multiply(overall_accuracy_pre).multiply(num_images_pre.subtract(mean_sum_prob_seagrass_pre)).divide(num_images_pre)

print(f"Uncertainty of pre-event seagrass area: {area_ha_pre:.2f} +- {uncertainty_pre_seagrass_area.getInfo():.2f} ha")

6.1.8 - Uncertainty analysis

In [ ]:
# ## To run uncertainties in the first classification (pre-event, 2018)
# ## t=(9*OA*(3-median probabilities))/3
# # 9 = area of pixel in square meters (3x3)
# # 3 = number of images

# ## first scale probabilities to 0-1 values
# def scale_to_unit(image):
#     return image.divide(100).copyProperties(image, image.propertyNames())

# # Apply the scaling function to the collection
# probabilities_01 = probabilities.map(scale_to_unit)

# median_probability = probabilities_01.select('seagrass').median().rename('seagrass_median_prob')
# median_probability

# # Select the band containing median probability
# median_probability = median_probability.select('seagrass_median_prob')

# # Apply the formula:
# # uncertainty = (9 * overall_accuracy * (3 - median_probability)) / 3
# three = ee.Number(3)
# nine = ee.Number(9)
# oa = ee.Number(overall_accuracy)

# # Compute: (3 - median_prob)
# diff = ee.Image.constant().subtract(median_probability)

# # Continue: (9 * overall_accuracy * diff) / 3
# # uncertainty = (diff.multiply(oa).multiply(nine)).divide(three)
# uncertainty = diff.divide(oa)

# # Add the uncertainty as a new band
# uncertainty_pre = image.addBands(uncertainty.rename('uncertainty'))

# region = uncertainty_pre.geometry()
# # Calculate min and max of the uncertainty band
# stats = uncertainty_pre.select('uncertainty').reduceRegion(
#     reducer=ee.Reducer.minMax(),
#     geometry=region,
#     scale=3,
#     maxPixels=1e9
# )

# # Get the results
# min_val = stats.get('uncertainty_min').getInfo()
# max_val = stats.get('uncertainty_max').getInfo()

# print('Uncertainty range:', min_val, 'to', max_val)


# # ###########################

# # Select the uncertainty band
# uncertainty_pre = uncertainty_pre.select('uncertainty')

# # Normalize to 0–1
# normalized_uncertainty = uncertainty.subtract(min_val).divide(max_val - min_val)

# # Add it back to the image
# overlap_mask = combined_classification_pre.mask().reduce(ee.Reducer.anyNonZero())
# uncertainty_pre_norm = image.addBands(normalized_uncertainty.rename('uncertainty_norm'))

# region = uncertainty_pre_norm.geometry()
# # Calculate min and max of the uncertainty band
# stats2 = uncertainty_pre_norm.select('uncertainty_norm').reduceRegion(
#     reducer=ee.Reducer.minMax(),
#     geometry=region,
#     scale=3,
#     maxPixels=1e9
# )

# # Get the results
# min_val = stats2.get('uncertainty_norm_min').getInfo()
# max_val = stats2.get('uncertainty_norm_max').getInfo()

# print('Uncertainty range norm:', min_val, 'to', max_val)

# # Visualize the mean uncertainty layer
# # Define visualization parameters for the uncertainty band
# # You may need to adjust the min/max values based on your data range
# vis_params = {'bands': ['uncertainty_norm'], 'palette': ['#4d004b', '#4f004d', '#50014e', '#520150', '#540251', '#550253', '#570354', '#580356', '#5a0457', '#5c0459', '#5d055a', '#5f055c', '#61065d', '#62065f', '#640761', '#650762', '#670864', '#690865', '#6a0867', '#6c0968', '#6e096a', '#6f0a6b', '#710a6d', '#730b6e', '#740b70', '#760c71', '#770c73', '#790d75', '#7b0d76', '#7c0e78', '#7e0e79', '#800f7b', '#810f7c', '#81117d', '#81127e', '#82147f', '#821580', '#821781', '#821982', '#831a83', '#831c84', '#831d85', '#831f86', '#832088', '#842289', '#84248a', '#84258b', '#84278c', '#85288d', '#852a8e', '#852b8f', '#852d90', '#852f91', '#863092', '#863293', '#863394', '#863595', '#873696', '#873897', '#873a98', '#873b99', '#873d9a', '#883e9b', '#88409c', '#88419d', '#88439e', '#88449e', '#88459f', '#8947a0', '#8948a0', '#8949a1', '#894ba2', '#894ca2', '#894da3', '#894fa3', '#8950a4', '#8a51a5', '#8a52a5', '#8a54a6', '#8a55a7', '#8a56a7', '#8a58a8', '#8a59a8', '#8a5aa9', '#8b5caa', '#8b5daa', '#8b5eab', '#8b60ac', '#8b61ac', '#8b62ad', '#8b64ad', '#8b65ae', '#8c66af', '#8c68af', '#8c69b0', '#8c6ab1', '#8c6cb1', '#8c6db2', '#8c6eb3', '#8c70b3', '#8c71b4', '#8c72b5', '#8c74b5', '#8c75b6', '#8c76b7', '#8c78b7', '#8c79b8', '#8c7ab8', '#8c7cb9', '#8c7dba', '#8c7eba', '#8c80bb', '#8c81bc', '#8c82bc', '#8c84bd', '#8c85be', '#8c86be', '#8c88bf', '#8c89c0', '#8c8bc0', '#8c8cc1', '#8c8dc2', '#8c8fc2', '#8c90c3', '#8c91c4', '#8c93c4', '#8c94c5', '#8c95c6', '#8c97c6', '#8d98c7', '#8d99c8', '#8e9ac8', '#8f9bc9', '#8f9dc9', '#909eca', '#909fcb', '#91a0cb', '#91a1cc', '#92a3cd', '#92a4cd', '#93a5ce', '#94a6ce', '#94a7cf', '#95a8d0', '#95aad0', '#96abd1', '#96acd2', '#97add2', '#98aed3', '#98b0d3', '#99b1d4', '#99b2d5', '#9ab3d5', '#9ab4d6', '#9bb6d7', '#9cb7d7', '#9cb8d8', '#9db9d9', '#9dbad9', '#9ebcda', '#9fbcda', '#a0bddb', '#a1bedb', '#a2bfdb', '#a3bfdc', '#a4c0dc', '#a5c1dc', '#a6c2dd', '#a7c2dd', '#a8c3de', '#a9c4de', '#aac4de', '#abc5df', '#acc6df', '#adc7e0', '#aec7e0', '#afc8e0', '#b0c9e1', '#b1c9e1', '#b2cae1', '#b3cbe2', '#b4cce2', '#b5cce3', '#b6cde3', '#b7cee3', '#b9cee4', '#bacfe4', '#bbd0e4', '#bcd1e5', '#bdd1e5', '#bed2e6', '#bfd3e6', '#c0d4e6', '#c1d4e7', '#c2d5e7', '#c3d6e8', '#c4d7e8', '#c5d8e9', '#c6d8e9', '#c7d9e9', '#c8daea', '#c9dbea', '#cadbeb', '#cbdceb', '#ccddec', '#cddeec', '#cedfec', '#cfdfed', '#d0e0ed', '#d1e1ee', '#d2e2ee', '#d3e2ef', '#d4e3ef', '#d6e4f0', '#d7e5f0', '#d8e6f0', '#d9e6f1', '#dae7f1', '#dbe8f2', '#dce9f2', '#ddeaf3', '#deeaf3', '#dfebf4', '#e0ecf4', '#e1ecf4', '#e1edf5', '#e2edf5', '#e3eef5', '#e4eef5', '#e4eff6', '#e5eff6', '#e6f0f6', '#e6f0f7', '#e7f1f7', '#e8f1f7', '#e9f2f7', '#e9f2f8', '#eaf3f8', '#ebf3f8', '#ebf4f8', '#ecf4f9', '#edf5f9', '#eef5f9', '#eef6fa', '#eff6fa', '#f0f7fa', '#f1f7fa', '#f1f8fb', '#f2f8fb', '#f3f9fb', '#f3f9fc', '#f4fafc', '#f5fafc', '#f6fbfc', '#f6fbfd', '#f7fcfd'], 'min': 0, 'max': 1}
# Map = geemap.Map()
# Map.addLayer(uncertainty_pre_norm.select('uncertainty_norm'), vis_params, "Uncertainty Pre-event")
# Map.centerObject(OSW, 13)
# Map

In [ ]:
# # Export the FeatureCollection to Google Drive
# task = ee.batch.Export.image.toDrive(
#     image=uncertainty_pre_norm.toFloat(),
#     description='uncertainty_pre_norm',  # Description for the task
#     folder='Colab_Notebooks/export',
#     region = OSW.geometry(),
#     scale=3,  # Adjust the scale as needed
#     fileFormat='GeoTIFF'  # Choose your desired file format (CSV, GeoJSON, KML, SHP, TFRecord)
# )

# # Start the export task
# task.start()

# print('Export task started. You can monitor its progress in the "Tasks" tab.')

In [ ]:
# ## Calculate uncertainty
# # probability IC: probabilities
# # classified IC: classification_images_pre
# # validation pre: v_FC

# def calculate_uncertainty(prob_image, classified_image, validation_fc):
#     # Get the probability bands from the probability image
#     probabilities_image = prob_image.select(['seagrass', 'nonSeagrass'])

#     # Sample the classified image with validation points
#     validated = classified_image.sampleRegions(
#         collection=validation_fc, # Using the provided validation FeatureCollection
#         properties=['habitat'], # Assuming 'habitat' is the class property in v_FC
#         scale=3,
#         geometries=True
#     )

#     # Calculate overall accuracy if there are enough validation points
#     overall_accuracy = ee.Algorithms.If(
#         validated.size().gt(1),
#         validated.errorMatrix('habitat', 'classification').accuracy(),
#         ee.Number(1.0) # Default accuracy if not enough points
#     )

#     # Ensure overall_accuracy is treated as an Earth Engine Number
#     overall_accuracy = ee.Number(overall_accuracy)

#     # Calculate uncertainty per pixel: (1 - Probability of Seagrass) / Overall Accuracy
#     # Ensure overall_accuracy is treated as a constant image for pixel-wise division
#     overall_accuracy_image = ee.Image.constant(overall_accuracy).float()

#     # Calculate the uncertainty based on seagrass probability
#     # Lower probability of seagrass indicates higher uncertainty for seagrass class
#     # A common approach is 1 - probability or using entropy. Let's use 1 - probability as a simple measure.
#     uncertainty_prob = probabilities_image.select('seagrass').multiply(-1).add(1).rename('uncertainty_prob')

#     # You could also consider scaling by accuracy, e.g., uncertainty_prob / overall_accuracy
#     uncertainty = uncertainty_prob.divide(overall_accuracy_image).rename('uncertainty')

#     # Return the uncertainty band, aligning it with the classified image properties
#     return classified_image.addBands(uncertainty).copyProperties(classified_image, classified_image.propertyNames())


# # Apply the function to your ImageCollections
# # We need to iterate through one collection (e.g., classification_images_pre) and find the corresponding image in the other (probabilities)
# def map_uncertainty_calculation(classified_image):
#     # Find the corresponding probability image using the system:index
#     prob_image = probabilities.filter(ee.Filter.eq('system:index', classified_image.get('system:index'))).first()

#     # Calculate uncertainty for this image pair
#     return calculate_uncertainty(prob_image, classified_image, v_FC)


# # Assuming 'classification_images_pre' is your pre-event ImageCollection containing the 'classification' band
# # Assuming 'probabilities' is your pre-event ImageCollection containing 'seagrass' and 'nonSeagrass' bands
# # Assuming 'v_FC' is your FeatureCollection of validation points (pre-event validation data)

# uncertainty_images_pre = soft_map.map(map_uncertainty_calculation())

# # You can now visualize or export the 'uncertainty' band from the uncertainty_images_pre collection
# print('Uncertainty calculation added to pre-event images.')

# # uncertainty_images_pre_mean = uncertainty_images_pre.mean()
# # uncertainty_images_pre_mean

# uncertainty_images_pre

In [ ]:
# # Display the combined classification
# Map.addLayer(uncertainty_images_pre.first().select('uncertainty'), {'min': 0, 'max': 1, 'palette': ['42762f', '7cb75a', 'e2e9ab', 'ffffff']}, 'Uncertainty')
# Map

## 6.2 - Post-event Multi-sensor Image Collection

In [ ]:
# Binary classification setup
seagrass_class = 0
nonSeagrass_class = 1

# Classification and validation parameters
trees = 50
t = 45

# Boxcar kernel: a square matrix where all values are equal, and it calculates the mean of the pixel values within that square neighborhood.
boxcar = ee.Kernel.square(radius=2, units='pixels', normalize=True)
bands = multi_sensor_post.bandNames()
# print(bands.getInfo())

def boxcar_image(image):
    return image.convolve(boxcar)

# Apply the function to every image in the collection
classificationComp_post = boxcar_image(multi_sensor_post)
classificationComp_post

#### 6.2.1 - Prepare training dataset

In [ ]:
# Training data - class property recoding and merging - MODEL 2 (POST-EVENT)
t_seagrass = t_seagrass_post.map(lambda x: x.setMulti(ee.Dictionary.fromLists(['habitat'], [seagrass_class])))
t_nonSeagrass = t_nonSeagrass_post.map(lambda x: x.setMulti(ee.Dictionary.fromLists(['habitat'], [nonSeagrass_class])))
t_FC = t_seagrass.merge(t_nonSeagrass)


# Collect sampled features from each image
def sample_image(image):
    sampled = image.sampleRegions(
        collection=t_FC,
        scale=3,
        geometries=True
    ).filter(ee.Filter.notNull(bands))

    return sampled

# Flatten the FeatureCollections into one big FeatureCollection
sampledData = sample_image(classificationComp_post)

# Using sampleRegions we take values across the Image Collection. getting valid FeatureCollection when overlapping with pixels
print('sampledData:', sampledData.size().getInfo())
print('t_FC:', t_FC.size().getInfo())

#### 6.2.2 - Extract probabilities for each class and train classifier

In [ ]:
## To extract feature importance and evaluate
def soft_prob_subfn(image, num):
       training = sampledData.map(lambda ft: ft.set(
        'prob',
        ee.Algorithms.If(ft.getNumber('habitat').eq(num), 1, 0)
        ))

       trained = ee.Classifier.smileRandomForest(numberOfTrees=trees) \
           .train(training, 'prob', image.bandNames()) \
           .setOutputMode('PROBABILITY')

       dict_classifier = trained.explain()

       # Create a Feature with the dictionary as a property
       feature = ee.Feature(None, {'classifier_explanation': dict_classifier})

       # Classify and convert to percentage scale
       classified = image.classify(trained).multiply(100).toInt8()

       # Return both the classified image and the Feature with explanation
       return ee.Image(classified).set('classifier_explanation', feature)

# Apply the function to the single image
seagrass_prob_image = soft_prob_subfn(classificationComp_post, seagrass_class)
nonSeagrass_prob_image = soft_prob_subfn(classificationComp_post, nonSeagrass_class)

seagrass_prob_image

# # Initialize an empty dictionary to store feature importances
# mean_importance = {}

# # Iterate through the first three images
# for i in range(3):
#   image = ee.Image(first_three_images.get(i))
#   classifier_explanation = image.get('classifier_explanation').getInfo()['properties']['classifier_explanation']
#   importance_dict = classifier_explanation.get('importance')

#   # Accumulate feature importances
#   for feature, importance in importance_dict.items():
#     mean_importance[feature] = mean_importance.get(feature, 0) + importance

# # Calculate the mean feature importance
# for feature in mean_importance:
#   mean_importance[feature] /= 3

# # Print the mean feature importance
# print(mean_importance)

# # Sort features by importance in descending order
# sorted_importance = dict(sorted(mean_importance.items(), key=lambda item: item[1], reverse=False))

# # Extract feature names and importance values
# feature_names = list(sorted_importance.keys())
# importance_values = list(sorted_importance.values())

# # Create a list of colors based on feature name prefix
# colors = ['#005b96' if name.startswith('PS_') else '#6497b1' for name in feature_names]

# # Create the horizontal bar graph (switched axes)
# plt.figure(figsize=(10, 6))  # Adjust figure size if needed
# plt.barh(feature_names, importance_values, color=colors) # Changed to barh for horizontal bars
# plt.ylabel("Features") # Switched x and y labels
# plt.xlabel("Importance") # Switched x and y labels
# plt.title("Feature Importance (2024)")
# # plt.axvline(x=4.2, linestyle='dotted', color='grey')  # Add dotted vertical line
# plt.show()  # Display the plot
# plt.tight_layout()

# selected_bands = [band for band, importance in mean_importance.items() if importance >= 0] ## Use only selected bands for the classification ##4.2 before

# print("Selected bands:", selected_bands)

In [ ]:
## Co-variance analysis
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

selected_bands = classificationComp_post.bandNames()

# Function to extract band values from a FeatureCollection
def extract_band_values(feature_collection, bands):
    data = feature_collection.toList(feature_collection.size()).map(lambda feature: ee.Feature(feature).toDictionary().select(bands)).getInfo()
    df = pd.DataFrame(data)
    return df

# Extract band values for selected bands
band_values_df = extract_band_values(sampledData, selected_bands)

# Calculate the correlation matrix
correlation_matrix = band_values_df.corr()

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, bool), k=1)

# Apply the mask to the filtered correlation matrix
filtered_correlation_matrix = correlation_matrix.mask(mask)
filtered_correlation_matrix = filtered_correlation_matrix[np.abs(correlation_matrix) > 0.80]

# Get bands with at least one correlation above 0.9
bands_to_keep = filtered_correlation_matrix.columns[filtered_correlation_matrix.any()]

# Filter the original correlation matrix to keep only relevant bands
filtered_correlation_matrix = correlation_matrix.loc[bands_to_keep, bands_to_keep]

# Plot only the lower triangle of the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(filtered_correlation_matrix, mask=mask, annot=True, cmap='coolwarm',
            fmt=".2f", linewidths=.5, linecolor='lightgrey', square=True,
            cbar_kws={"shrink": .75})
plt.title('Band Covariability Analysis')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("Bands to keep:", bands_to_keep.tolist())

In [ ]:
# #### REMOVE BANDS THAT ARE NOT USEFUL

# # Define the bands to remove
# bands_to_remove = ['PS_B8', 'S2_B8', 'S2_B6'] # 'PS_Kd_505', 'S2_B6', 'S2_B7', 'S2_B8',

# # Create a list of bands to keep by filtering out the bands to remove
# selected_bands = [band for band in selected_bands if band not in bands_to_remove]

# # Apply the select method to the Image Collection to keep the desired bands
# classificationComp_post = classificationComp_post.select(selected_bands)
# classificationComp_post.first().bandNames()
# classificationComp_post

### 6.2.3 - Generate probability layers

In [ ]:
## Train classifier and display probability map for seagrasses
# Define classes dictionary
classes = {
  'classes_values': [seagrass_class, nonSeagrass_class],
  'classes_names': ['seagrass','nonSeagrass']
}

classes

# Assume this is already defined:
# - classificationComp_pre: an ImageCollection
# - classes: dictionary with class values and names
# - soft_prob_subfn(image, class_id): returns a probability image for one class

def classify_with_all_classes(image):
    # Classify image for each class and rename the result accordingly
    classified_images = ee.List(classes['classes_values']).map(
        lambda class_id: soft_prob_subfn(image, class_id)
    )

    # Convert list of images into one multi-band image (each class as band)
    classified_combined = ee.ImageCollection(classified_images).toBands()
    classified_combined = classified_combined.rename(classes['classes_names'])

    # Preserve metadata if needed
    return classified_combined.copyProperties(image, image.propertyNames())

# Generate probabilities in the ImageCollection
probabilities = classify_with_all_classes(classificationComp_post)
probabilities

In [ ]:
# Visualize probabilities
probabilities = ee.Image(probabilities).clip(OSW).updateMask(land_mask)
Map = geemap.Map()
Map.addLayer(probabilities.select('seagrass'), {'min': 45, 'max': 100}, 'Seagrass Prob')
Map.addLayer(probabilities.select('nonSeagrass'), {'min': 45, 'max': 100}, 'NonSeagrass Prob')
Map.centerObject(probabilities, zoom=12)
Map

#### 6.2.4 - Threshold classes based on probabilities and classify

In [ ]:
# Based on the generated probabilities, apply thresholding for seagrass and nonSeagrass on the whole Image Collection

# Threshold function for seagrass
def threshold_seagrass(image):
    return image.select('seagrass').updateMask(image.select('seagrass').gt(t)) \
        .copyProperties(image, image.propertyNames())

# Threshold function for non-seagrass
def threshold_non_seagrass(image):
    return image.select('nonSeagrass').updateMask(image.select('nonSeagrass').gt(t)) \
        .copyProperties(image, image.propertyNames())

# Map the thresholding over the probabilities collection
seagrass_soft = threshold_seagrass(probabilities)
nonSeagrass_soft = threshold_non_seagrass(probabilities)

# Combine the thresholded images into a single image (taking the first image from each collection as an example)
# If you intend to process all images in the collection, the downstream code needs to be adapted to work with ImageCollections.
soft_map = ee.Image(seagrass_soft).addBands(ee.Image(nonSeagrass_soft))
soft_map

#### 6.2.5 - Perform classification

In [ ]:
def soft_to_hard(image):
    seagrass = image.select('seagrass')
    non_seagrass = image.select('nonSeagrass')
    prob_aoi = ee.Image(probabilities).geometry()

    classified = (ee.Image.constant(2)
        .where(seagrass.gte(t), ee.Number(0).add(1))  # temporary +1 to mask 0s
        .where(non_seagrass.gte(t), ee.Number(1).add(1))
        .selfMask()
        .subtract(1)  # back to 0 and 1
        .rename('classification')
        .clip(prob_aoi))
    return classified

# Apply to soft_map
soft_map_th = soft_to_hard(soft_map)
soft_map_th

#### 6.2.6 - Combine the three classifications for visualization

In [ ]:
# 1. Extract the 'classification' band from each image
classification_images_post = soft_map_th.select('classification').mask(land_mask).clip(OSW)

# Display the combined classification
Map.addLayer(classification_images_post, {'min': 0, 'max': 1, 'palette': ['42762f', '7cb75a', 'e2e9ab', 'ffffff']}, 'Combined Classification')
Map

In [ ]:
# # Export the FeatureCollection to Google Drive
# task = ee.batch.Export.image.toDrive(
#     image=combined_classification_post,
#     description='combined_classification_post',  # Description for the task
#     folder='Colab_Notebooks/export',
#     region = OSW.geometry(),
#     scale=3,  # Adjust the scale as needed
#     fileFormat='GeoTIFF'  # Choose your desired file format (CSV, GeoJSON, KML, SHP, TFRecord)
# )

# # Start the export task
# task.start()

# print('Export task started. You can monitor its progress in the "Tasks" tab.')

In [ ]:
# Apply conditional classification for obtain the final binary map
classification_images_post = classification_images_post.expression(
    'b(0) < 0.35 ? 0 : 1',  # Conditional expression
    {'classification': classification_images_post}  # Band name mapping
).rename('classification')  # Rename the band

# Display the combined classification
Map.addLayer(classification_images_post.mask(land_mask).clip(OSW.geometry()), {'min': 0, 'max': 1, 'palette': ['42762f', 'ffffff']}, 'Combined Classification')
Map

#### 6.2.7 - Validation and accuracy assessment for each of the 3 classifications

In [ ]:
# Validation function
def valid_habitat(number):
  def wrap(feature):
    return feature.setMulti(ee.Dictionary.fromLists(['habitat'], [number]))
  return wrap

## Validation data
v_seagrass = v_seagrass_post.map(valid_habitat(seagrass_class))
v_nonSeagrass = v_nonSeagrass_post.map(valid_habitat(nonSeagrass_class))
v_FC = v_seagrass.merge(v_nonSeagrass)
v_FC

In [ ]:
## To validate with the final combination of the 3 classifications (this will be better)
# sampledValidation_post = final_classification_post.sampleRegions(
#     collection=v_FC,
#     scale=3,
#     properties=['habitat'])

# sampledValidation_post

In [ ]:
# Collect sampled features from each image
def validation(image):
    # Buffer the validation points by 10 meters (optional, keep if desired)
   # buffered_v_FC = v_FC.map(lambda feature: feature.buffer(5))

    sampled = image.sampleRegions(
        collection=v_FC,
        properties=['habitat'],
        scale=3
        )
    return sampled


# Map the sampling function and convert result to a list of FeatureCollections
sampled_v_list = validation(soft_map_th)

# Flatten the FeatureCollections into one big FeatureCollection
sampledValidation_post = ee.FeatureCollection(sampled_v_list) #.filter(ee.Filter.notNull(['habitat']))

## To validate with the final combination of the 3 classifications (this will be better)
# sampledValidation_pre = final_classification_pre.sampleRegions(
#     collection=v_FC,
#     scale=3,
#     properties=['habitat'])

# Using sampleRegions we take values across the Image Collection. getting valid FeatureCollection when overlapping with pixels
print('sampled Validation:', sampledValidation_post.size().getInfo())
sampledValidation_post

In [ ]:
errorMatrix = sampledValidation_post.errorMatrix('habitat', 'classification')
print('Confusion Matrix:', errorMatrix.getInfo())

# Get confusion matrix data as a list of lists
confusion_matrix_data = errorMatrix.array().getInfo()

# Convert the list to a NumPy array
matrix_values = np.array(confusion_matrix_data)  # Convert to NumPy array

# Define class labels
class_names = ['seagrass', 'nonSeagrass']  # Adjust if your class names are different

# Create ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(confusion_matrix=matrix_values, display_labels=class_names)

# Plot the confusion matrix
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.xlabel("EO prediction", fontsize = 14)  # Change to your desired text
plt.ylabel("In situ data", fontsize = 14)
disp.ax_.tick_params(axis='both', which='major', labelsize=18)  # Increase axis tick label size  # Change to your desired text
plt.show()

# Print accuracy metrics
overall_accuracy = errorMatrix.accuracy().getInfo()
print('Overall Accuracy:', overall_accuracy)
producers_accuracy = ee.Array(errorMatrix.producersAccuracy()).reshape([-1]).getInfo()
print('PA:', producers_accuracy)
users_accuracy = ee.Array(errorMatrix.consumersAccuracy()).reshape([-1]).getInfo()
print('UA:', users_accuracy)

# Calcuate F1-score
# Extract the components from the confusion matrix
truePositives = errorMatrix.array().get([0, 0])
falsePositives = errorMatrix.array().get([1, 0])
falseNegatives = errorMatrix.array().get([0, 1])

# Calculate precision, recall, and F1-score
precision = truePositives.divide(truePositives.add(falsePositives))
recall = truePositives.divide(truePositives.add(falseNegatives))
f1_score = ee.Number(2).multiply(precision.multiply(recall)).divide(precision.add(recall))

print('F1-score:', f1_score.getInfo())

In [ ]:
## Calculate hectares
seagrass_post = classification_images_post.select('classification').eq(0)

# Step 3: Count number of seagrass pixels
pixel_stats = seagrass_post.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=OSW,
    scale=3,
    maxPixels=1e13
)

# Step 4: Convert to hectares (each 3x3m pixel = 9 m² = 0.0009 ha)
seagrass_pixel_count = ee.Number(pixel_stats.get('classification'))
hectares = seagrass_pixel_count.multiply(0.0009)

# Print the result
print('Seagrass area (hectares):', hectares.getInfo())

In [ ]:
# ## Calculate hectares
# seagrass_pre = classification_images_pre.select('classification').eq(0)

# # Count number of seagrass pixels
# pixel_stats = seagrass_pre.reduceRegion(
#     reducer=ee.Reducer.sum(),
#     geometry=OSW,
#     scale=3,
#     maxPixels=1e13
# )

# # Step 4: Convert to hectares (each 3x3m pixel = 9 m² = 0.0009 ha)
# seagrass_pixel_count = ee.Number(pixel_stats.get('classification'))
# hectares = seagrass_pixel_count.multiply(0.0009)

# # Print the result
# print('Seagrass area (hectares):', hectares.getInfo())

### Area quantification and +-

In [ ]:
# Mask seagrass pixels
seagrass_post = classification_images_post.eq(0)

# create a binary mask: 1 = seagrass, 0 = not
seagrass_mask = seagrass_post.rename('seagrass')  # change eq(0) if your seagrass label differs

# count seagrass pixels (sum of 1s)
pixel_count = seagrass_mask.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=OSW.geometry(),
    scale=3, # 3 m pixels
    maxPixels=1e13
).get('seagrass').getInfo()

# convert to hectares: each pixel = 9 m2 -> 9/10000 = 0.0009 ha
area_ha_post = pixel_count * 9.0 / 10000.0

print(f"Seagrass pixels: {pixel_count}, area: {area_ha_post} ha")

In [ ]:
# Scale probabilities from 0-100 to 0-1
probabilities_post = probabilities.divide(100).select('seagrass')

# Get the mean probability of seagrass over the region of interest
mean_sum_prob_seagrass_post = probabilities_post.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=OSW.geometry(),
    scale=3, # Use the same scale as the area calculation
    maxPixels=1e13
).get('seagrass') # Get the mean value for the 'seagrass' band

overall_accuracy_post = ee.Number(overall_accuracy) # Use the overall_accuracy from the pre-event validation cell

# Get the total seagrass area for the pre-event classification (assuming it's stored in area_ha variable from cell ZfHt_UQ49oKf)
# If not, you might need to recalculate it here or fetch it from the appropriate variable.
total_seagrass_area_post = ee.Number(area_ha_post)

# Calculate uncertainty for the pre-event seagrass area using the provided formula:
# uncertainty = area * Overall Accuracy * (3 - mean of the sum of the probabilities predicted as seagrass) / 3
# Assuming '3' is the number of images used in the pre-event period (sentinel2_l1c_pre has 3 images)
num_images_post = ee.Number(1) # Set the number of pre-event images to 1 as an Earth Engine Number

uncertainty_post_seagrass_area = total_seagrass_area_post.multiply(overall_accuracy_post).multiply(num_images_post.subtract(mean_sum_prob_seagrass_post)).divide(num_images_post)

print(f"Uncertainty of pre-event seagrass area: {area_ha_post:.2f} +- {uncertainty_post_seagrass_area.getInfo():.2f} ha")

#### 6.2.8 - Uncertainty analysis

In [ ]:
# ## To run uncertainties in the first classification (pre-event, 2018)
# ## t=(9*OA*(3-median probabilities))/3
# # 9 = area of pixel in square meters (3x3)
# # 3 = number of images

# ## first scale probabilities to 0-1 values
# def scale_to_unit(image):
#     return image.divide(100).copyProperties(image, image.propertyNames())

# # Apply the scaling function to the collection
# probabilities_01 = probabilities.map(scale_to_unit)

# median_probability = probabilities_01.select('seagrass').median().rename('seagrass_median_prob')
# median_probability

# # Select the band containing median probability
# median_probability = median_probability.select('seagrass_median_prob')

# # Apply the formula:
# # uncertainty = (9 * overall_accuracy * (3 - median_probability)) / 3
# three = ee.Number(3)
# nine = ee.Number(9)
# oa = ee.Number(overall_accuracy)

# # Compute: (3 - median_prob)
# diff = ee.Image.constant(1).subtract(median_probability)

# # Continue: (9 * overall_accuracy * diff) / 3
# # uncertainty = (diff.multiply(oa).multiply(nine)).divide(three)
# uncertainty = diff.divide(oa)

# # Add the uncertainty as a new band
# uncertainty_post = image.addBands(uncertainty.rename('uncertainty'))

# region = uncertainty_post.geometry()
# # Calculate min and max of the uncertainty band
# stats = uncertainty_post.select('uncertainty').reduceRegion(
#     reducer=ee.Reducer.minMax(),
#     geometry=region,
#     scale=3,
#     maxPixels=1e9
# )

# # Get the results
# min_val = stats.get('uncertainty_min').getInfo()
# max_val = stats.get('uncertainty_max').getInfo()

# print('Uncertainty range:', min_val, 'to', max_val)


# # ###########################

# # Select the uncertainty band
# uncertainty_post = uncertainty_post.select('uncertainty')

# # Normalize to 0–1
# normalized_uncertainty = uncertainty_post.subtract(min_val).divide(max_val - min_val)

# # Add it back to the image
# overlap_mask = combined_classification_post.mask().reduce(ee.Reducer.anyNonZero())
# uncertainty_post_norm = image.addBands(normalized_uncertainty.rename('uncertainty_norm'))

# region = uncertainty_pre_norm.geometry()
# # Calculate min and max of the uncertainty band
# stats2 = uncertainty_post_norm.select('uncertainty_norm').reduceRegion(
#     reducer=ee.Reducer.minMax(),
#     geometry=region,
#     scale=3,
#     maxPixels=1e9
# )

# # Get the results
# min_val = stats2.get('uncertainty_norm_min').getInfo()
# max_val = stats2.get('uncertainty_norm_max').getInfo()

# print('Uncertainty range norm:', min_val, 'to', max_val)

# # Visualize the mean uncertainty layer
# # Define visualization parameters for the uncertainty band
# # You may need to adjust the min/max values based on your data range
# vis_params = {'bands': ['uncertainty_norm'], 'palette': ['#4d004b', '#4f004d', '#50014e', '#520150', '#540251', '#550253', '#570354', '#580356', '#5a0457', '#5c0459', '#5d055a', '#5f055c', '#61065d', '#62065f', '#640761', '#650762', '#670864', '#690865', '#6a0867', '#6c0968', '#6e096a', '#6f0a6b', '#710a6d', '#730b6e', '#740b70', '#760c71', '#770c73', '#790d75', '#7b0d76', '#7c0e78', '#7e0e79', '#800f7b', '#810f7c', '#81117d', '#81127e', '#82147f', '#821580', '#821781', '#821982', '#831a83', '#831c84', '#831d85', '#831f86', '#832088', '#842289', '#84248a', '#84258b', '#84278c', '#85288d', '#852a8e', '#852b8f', '#852d90', '#852f91', '#863092', '#863293', '#863394', '#863595', '#873696', '#873897', '#873a98', '#873b99', '#873d9a', '#883e9b', '#88409c', '#88419d', '#88439e', '#88449e', '#88459f', '#8947a0', '#8948a0', '#8949a1', '#894ba2', '#894ca2', '#894da3', '#894fa3', '#8950a4', '#8a51a5', '#8a52a5', '#8a54a6', '#8a55a7', '#8a56a7', '#8a58a8', '#8a59a8', '#8a5aa9', '#8b5caa', '#8b5daa', '#8b5eab', '#8b60ac', '#8b61ac', '#8b62ad', '#8b64ad', '#8b65ae', '#8c66af', '#8c68af', '#8c69b0', '#8c6ab1', '#8c6cb1', '#8c6db2', '#8c6eb3', '#8c70b3', '#8c71b4', '#8c72b5', '#8c74b5', '#8c75b6', '#8c76b7', '#8c78b7', '#8c79b8', '#8c7ab8', '#8c7cb9', '#8c7dba', '#8c7eba', '#8c80bb', '#8c81bc', '#8c82bc', '#8c84bd', '#8c85be', '#8c86be', '#8c88bf', '#8c89c0', '#8c8bc0', '#8c8cc1', '#8c8dc2', '#8c8fc2', '#8c90c3', '#8c91c4', '#8c93c4', '#8c94c5', '#8c95c6', '#8c97c6', '#8d98c7', '#8d99c8', '#8e9ac8', '#8f9bc9', '#8f9dc9', '#909eca', '#909fcb', '#91a0cb', '#91a1cc', '#92a3cd', '#92a4cd', '#93a5ce', '#94a6ce', '#94a7cf', '#95a8d0', '#95aad0', '#96abd1', '#96acd2', '#97add2', '#98aed3', '#98b0d3', '#99b1d4', '#99b2d5', '#9ab3d5', '#9ab4d6', '#9bb6d7', '#9cb7d7', '#9cb8d8', '#9db9d9', '#9dbad9', '#9ebcda', '#9fbcda', '#a0bddb', '#a1bedb', '#a2bfdb', '#a3bfdc', '#a4c0dc', '#a5c1dc', '#a6c2dd', '#a7c2dd', '#a8c3de', '#a9c4de', '#aac4de', '#abc5df', '#acc6df', '#adc7e0', '#aec7e0', '#afc8e0', '#b0c9e1', '#b1c9e1', '#b2cae1', '#b3cbe2', '#b4cce2', '#b5cce3', '#b6cde3', '#b7cee3', '#b9cee4', '#bacfe4', '#bbd0e4', '#bcd1e5', '#bdd1e5', '#bed2e6', '#bfd3e6', '#c0d4e6', '#c1d4e7', '#c2d5e7', '#c3d6e8', '#c4d7e8', '#c5d8e9', '#c6d8e9', '#c7d9e9', '#c8daea', '#c9dbea', '#cadbeb', '#cbdceb', '#ccddec', '#cddeec', '#cedfec', '#cfdfed', '#d0e0ed', '#d1e1ee', '#d2e2ee', '#d3e2ef', '#d4e3ef', '#d6e4f0', '#d7e5f0', '#d8e6f0', '#d9e6f1', '#dae7f1', '#dbe8f2', '#dce9f2', '#ddeaf3', '#deeaf3', '#dfebf4', '#e0ecf4', '#e1ecf4', '#e1edf5', '#e2edf5', '#e3eef5', '#e4eef5', '#e4eff6', '#e5eff6', '#e6f0f6', '#e6f0f7', '#e7f1f7', '#e8f1f7', '#e9f2f7', '#e9f2f8', '#eaf3f8', '#ebf3f8', '#ebf4f8', '#ecf4f9', '#edf5f9', '#eef5f9', '#eef6fa', '#eff6fa', '#f0f7fa', '#f1f7fa', '#f1f8fb', '#f2f8fb', '#f3f9fb', '#f3f9fc', '#f4fafc', '#f5fafc', '#f6fbfc', '#f6fbfd', '#f7fcfd'], 'min': 0, 'max': 1}
# Map = geemap.Map()
# Map.addLayer(uncertainty_post_norm.select('uncertainty_norm'), vis_params, "Uncertainty Post-event")
# Map.centerObject(OSW, 13)
# Map

In [ ]:
# # Export the FeatureCollection to Google Drive
# task = ee.batch.Export.image.toDrive(
#     image=uncertainty_post_norm.toFloat(),
#     description='uncertainty_post_norm',  # Description for the task
#     folder='Colab_Notebooks/export',
#     region = OSW.geometry(),
#     scale=3,  # Adjust the scale as needed
#     fileFormat='GeoTIFF'  # Choose your desired file format (CSV, GeoJSON, KML, SHP, TFRecord)
# )

# # Start the export task
# task.start()

# print('Export task started. You can monitor its progress in the "Tasks" tab.')

# 7 - Seagrass change detection

In [ ]:
seagrass_pre = final_classification_pre.updateMask(final_classification_pre.select('classification').eq(0))
seagrass_post = final_classification_post.updateMask(final_classification_post.select('classification').eq(0))

seagrass_pre

In [ ]:
# Change from 0 to 1 (e.g., newly appeared features)
gain = final_classification_pre.eq(1).And(final_classification_post.eq(0))

# Change from 1 to 0 (e.g., disappeared features)
loss = final_classification_pre.eq(0).And(final_classification_post.eq(1))

# Stable seagrass meadows
stable = final_classification_pre.eq(0).And(final_classification_post.eq(0))

# Combine the conditions into a single image using 'where'
seagrass_change = ee.Image(0) \
    .where(loss, 1) \
    .where(stable, 2) \
    .where(gain, 3)

seagrass_change

In [ ]:
# Display the combined classification
seagrass_change = seagrass_change.updateMask(land_mask).clip(OSW).selfMask()
Map.addLayer(seagrass_change, {'min': 1, 'max': 3, 'palette': ['df8556', '42762f', '67688f']}, 'Combined Classification')
Map

In [ ]:
# Define the export task
task = ee.batch.Export.image.toDrive(
    image=seagrass_change,
    description='seagrass_change',  # Description for the task
    folder='Colab_Notebooks/export',  # Folder in Google Drive to save the image (change if needed)
    fileNamePrefix='seagrass_change',  # Prefix for the file name
    region=roi,  # Region of interest (assuming 'roi' is defined)
    scale=3,  # Scale in meters (adjust if needed)
    fileFormat='GeoTIFF',  # File format
    maxPixels=1e13  # Maximum number of pixels allowed (adjust if needed)
)

# Start the export task
task.start()
print('Export task started. You can monitor its progress in the "Tasks" tab.')

In [ ]:
# # Visualize desirable bands to check data
# s2_b8a_pre = multi_sensor_pre.first().select('S2_chl_re_mishra')

# # Define visualization parameters for S2_B8A
# # Adjust min/max values based on your data range
# s2_b8a_vis = {
#     'min': 0,
#     'max': 0.15,  # Example max value, adjust as needed
#     'palette': 'viridis' # Or any other suitable palette
# }

# Map.addLayer(s2_b8a_pre, s2_b8a_vis, "Sentinel-2 chla")
# Map

Map.addLayer(seagrass_samples_18, {'color': 'darkgreen'}, 'in situ seagrass 2018')
Map.addLayer(nonSeagrass_samples_18, {'color': 'orange'}, 'in situ nonSeagrass 2018')

Map.addLayer(sampledValidation_pre, {'color': 'pink'}, 'sampledValidation_pre')

Map.addLayer(seagrass_samples_24, {'color': 'lightgreen'}, 'in situ seagrass 2024')
Map.addLayer(nonSeagrass_samples_24, {'color': 'yellow'}, 'in situ nonSeagrass 2024')
Map

In [ ]:
# Select bands
pre = uncertainty_pre_norm.select('uncertainty_norm')
post = uncertainty_post_norm.select('uncertainty_norm')

# Root-sum-of-squares
uncertainty_change = pre.pow(2).add(post.pow(2)).sqrt().rename('uncertainty_change')

# Optionally normalize or add to a base image
result = image.addBands(uncertainty_change)
result

In [ ]:
region = OSW.geometry()
# Calculate min and max of the uncertainty band
stats2 = uncertainty_change.select('uncertainty_change').reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=region,
    scale=3,
    maxPixels=1e9
)

# Get the results
min_val = stats2.get('uncertainty_change_min').getInfo()
max_val = stats2.get('uncertainty_change_max').getInfo()

print('Uncertainty range:', min_val, 'to', max_val)

# Select the uncertainty band
uncertainty_change = uncertainty_change.select('uncertainty_change')

# Normalize to 0–1
normalized_uncertainty = uncertainty_change.subtract(min_val).divide(max_val - min_val)

# Add it back to the image
overlap_mask = uncertainty_change.mask().reduce(ee.Reducer.anyNonZero())
uncertainty_change_norm = image.addBands(normalized_uncertainty.rename('uncertainty_change_norm'))

region = uncertainty_pre_norm.geometry()
# Calculate min and max of the uncertainty band
stats2 = uncertainty_post_norm.select('uncertainty_change_norm').reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=region,
    scale=3,
    maxPixels=1e9
)

# Get the results
min_val = stats2.get('uncertainty_change_norm_min').getInfo()
max_val = stats2.get('uncertainty_change_norm_max').getInfo()

print('Uncertainty range norm:', min_val, 'to', max_val)


# Visualize the mean uncertainty layer
# Define visualization parameters for the uncertainty band
# You may need to adjust the min/max values based on your data range
vis_params = {'bands': ['uncertainty_change_norm'], 'palette': ['#4d004b', '#4f004d', '#50014e', '#520150', '#540251', '#550253', '#570354', '#580356', '#5a0457', '#5c0459', '#5d055a', '#5f055c', '#61065d', '#62065f', '#640761', '#650762', '#670864', '#690865', '#6a0867', '#6c0968', '#6e096a', '#6f0a6b', '#710a6d', '#730b6e', '#740b70', '#760c71', '#770c73', '#790d75', '#7b0d76', '#7c0e78', '#7e0e79', '#800f7b', '#810f7c', '#81117d', '#81127e', '#82147f', '#821580', '#821781', '#821982', '#831a83', '#831c84', '#831d85', '#831f86', '#832088', '#842289', '#84248a', '#84258b', '#84278c', '#85288d', '#852a8e', '#852b8f', '#852d90', '#852f91', '#863092', '#863293', '#863394', '#863595', '#873696', '#873897', '#873a98', '#873b99', '#873d9a', '#883e9b', '#88409c', '#88419d', '#88439e', '#88449e', '#88459f', '#8947a0', '#8948a0', '#8949a1', '#894ba2', '#894ca2', '#894da3', '#894fa3', '#8950a4', '#8a51a5', '#8a52a5', '#8a54a6', '#8a55a7', '#8a56a7', '#8a58a8', '#8a59a8', '#8a5aa9', '#8b5caa', '#8b5daa', '#8b5eab', '#8b60ac', '#8b61ac', '#8b62ad', '#8b64ad', '#8b65ae', '#8c66af', '#8c68af', '#8c69b0', '#8c6ab1', '#8c6cb1', '#8c6db2', '#8c6eb3', '#8c70b3', '#8c71b4', '#8c72b5', '#8c74b5', '#8c75b6', '#8c76b7', '#8c78b7', '#8c79b8', '#8c7ab8', '#8c7cb9', '#8c7dba', '#8c7eba', '#8c80bb', '#8c81bc', '#8c82bc', '#8c84bd', '#8c85be', '#8c86be', '#8c88bf', '#8c89c0', '#8c8bc0', '#8c8cc1', '#8c8dc2', '#8c8fc2', '#8c90c3', '#8c91c4', '#8c93c4', '#8c94c5', '#8c95c6', '#8c97c6', '#8d98c7', '#8d99c8', '#8e9ac8', '#8f9bc9', '#8f9dc9', '#909eca', '#909fcb', '#91a0cb', '#91a1cc', '#92a3cd', '#92a4cd', '#93a5ce', '#94a6ce', '#94a7cf', '#95a8d0', '#95aad0', '#96abd1', '#96acd2', '#97add2', '#98aed3', '#98b0d3', '#99b1d4', '#99b2d5', '#9ab3d5', '#9ab4d6', '#9bb6d7', '#9cb7d7', '#9cb8d8', '#9db9d9', '#9dbad9', '#9ebcda', '#9fbcda', '#a0bddb', '#a1bedb', '#a2bfdb', '#a3bfdc', '#a4c0dc', '#a5c1dc', '#a6c2dd', '#a7c2dd', '#a8c3de', '#a9c4de', '#aac4de', '#abc5df', '#acc6df', '#adc7e0', '#aec7e0', '#afc8e0', '#b0c9e1', '#b1c9e1', '#b2cae1', '#b3cbe2', '#b4cce2', '#b5cce3', '#b6cde3', '#b7cee3', '#b9cee4', '#bacfe4', '#bbd0e4', '#bcd1e5', '#bdd1e5', '#bed2e6', '#bfd3e6', '#c0d4e6', '#c1d4e7', '#c2d5e7', '#c3d6e8', '#c4d7e8', '#c5d8e9', '#c6d8e9', '#c7d9e9', '#c8daea', '#c9dbea', '#cadbeb', '#cbdceb', '#ccddec', '#cddeec', '#cedfec', '#cfdfed', '#d0e0ed', '#d1e1ee', '#d2e2ee', '#d3e2ef', '#d4e3ef', '#d6e4f0', '#d7e5f0', '#d8e6f0', '#d9e6f1', '#dae7f1', '#dbe8f2', '#dce9f2', '#ddeaf3', '#deeaf3', '#dfebf4', '#e0ecf4', '#e1ecf4', '#e1edf5', '#e2edf5', '#e3eef5', '#e4eef5', '#e4eff6', '#e5eff6', '#e6f0f6', '#e6f0f7', '#e7f1f7', '#e8f1f7', '#e9f2f7', '#e9f2f8', '#eaf3f8', '#ebf3f8', '#ebf4f8', '#ecf4f9', '#edf5f9', '#eef5f9', '#eef6fa', '#eff6fa', '#f0f7fa', '#f1f7fa', '#f1f8fb', '#f2f8fb', '#f3f9fb', '#f3f9fc', '#f4fafc', '#f5fafc', '#f6fbfc', '#f6fbfd', '#f7fcfd'], 'min': 0, 'max': 1}
Map = geemap.Map()
Map.addLayer(uncertainty_change_norm.select('uncertainty_change_norm'), vis_params, "Uncertainty change")
Map.centerObject(OSW, 13)
Map

In [ ]:
# # Define the export task
# task = ee.batch.Export.image.toDrive(
#     image=uncertainty_change_norm.toFloat(),
#     description='uncertainty_change_norm',  # Description for the task
#     folder='Colab_Notebooks/export',  # Folder in Google Drive to save the image (change if needed)
#     fileNamePrefix='uncertainty_change_norm',  # Prefix for the file name
#     region=roi,  # Region of interest (assuming 'roi' is defined)
#     scale=3,  # Scale in meters (adjust if needed)
#     fileFormat='GeoTIFF',  # File format
#     maxPixels=1e13  # Maximum number of pixels allowed (adjust if needed)
# )

# # Start the export task
# task.start()
# print('Export task started. You can monitor its progress in the "Tasks" tab.')

In [ ]:
## To extract feature importance and evaluate for PlanetScope 2018
def soft_prob_subfn_ps18(image, num):
       training = sampledData.map(lambda ft: ft.set(
        'prob',
        ee.Algorithms.If(ft.getNumber('habitat').eq(num), 1, 0)
        ))

       trained = ee.Classifier.smileRandomForest(numberOfTrees=trees) \
           .train(training, 'prob', image.bandNames()) \
           .setOutputMode('PROBABILITY')

       dict_classifier = trained.explain()

       # Create a Feature with the dictionary as a property
       feature = ee.Feature(None, {'classifier_explanation': dict_classifier})

       # Classify and convert to percentage scale
       classified = image.classify(trained).multiply(100).toInt8()

       # Return both the classified image and the Feature with explanation
       return ee.Image(classified).set('classifier_explanation', feature)

# Apply the function to the single PlanetScope 2018 image
seagrass_prob_image_ps18 = soft_prob_subfn_ps18(classificationComp_pre, seagrass_class)

### FEATURE IMPORTANCE FOR PLANETSCOPE 2018
# Get the classifier explanation from the single image
classifier_explanation_ps18 = seagrass_prob_image_ps18.get('classifier_explanation').getInfo()['properties']['classifier_explanation']
importance_dict_ps18 = classifier_explanation_ps18.get('importance')

# Sort features by importance in descending order
sorted_importance_ps18 = dict(sorted(importance_dict_ps18.items(), key=lambda item: item[1], reverse=False))

# Extract feature names and importance values
feature_names_ps18 = list(sorted_importance_ps18.keys())
importance_values_ps18 = list(sorted_importance_ps18.values())

# Create a list of colors based on feature name prefix
colors_ps18 = ['#005b96' if name.startswith('PS_') else '#6497b1' for name in feature_names_ps18]

# Create the horizontal bar graph (switched axes)
plt.figure(figsize=(10, 6))  # Adjust figure size if needed
plt.barh(feature_names_ps18, importance_values_ps18, color=colors_ps18) # Changed to barh for horizontal bars
plt.ylabel("Features") # Switched x and y labels
plt.xlabel("Importance") # Switched x and y labels
plt.title("Feature Importance (2018)")
plt.show()  # Display the plot
plt.tight_layout()

selected_bands_ps18 = [band for band, importance in importance_dict_ps18.items() if importance >= 0]

print("Selected bands (2018):", selected_bands_ps18)